In [1]:
from importlib import reload

import datasets
import numpy as np

import src.models.components.spatiotemporal_encoder
import src.models.components.positional_encoding
import src.models.components.feature_extractor_dinov2

In [4]:
rwth_phoenix = datasets.load_dataset('lukasbraach/rwth_phoenix_weather_2014', 'multisigner', streaming=True)
it = iter(rwth_phoenix['train'])
first = next(it)

In [4]:
reload(src.models.components.feature_extractor_dinov2)
from src.models.components.feature_extractor_dinov2 import SignLanguageFeatureExtractor

feature_extractor = SignLanguageFeatureExtractor()

src_tensor = [
    np.random.rand(3, 3, 224, 224),
    np.random.rand(1, 3, 224, 224),
    np.random.rand(6, 3, 224, 224),
]
batched_feature = feature_extractor(src_tensor, sampling_rate=25, return_tensors='pt', return_attention_mask=True, padding=True)


print("input_values:", batched_feature['input_values'].shape)
print("attention_mask:", batched_feature['attention_mask'])

input_values: torch.Size([3, 6, 768])
attention_mask: tensor([[1., 1., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1.]], dtype=torch.float64)


In [5]:
reload(src.models.components.positional_encoding)
reload(src.models.components.spatiotemporal_encoder)
from src.models.components.spatiotemporal_encoder import SpatiotemporalEncoder


model = SpatiotemporalEncoder()
encoded = model(batched_feature)

encoded['input_values']

RuntimeError: Batching rule not implemented for aten::equal. We could not generate a fallback.

In [2]:
from src.models.components.sign_language_net import SignLanguageNet
import torch

model = SignLanguageNet()

src_tensor = torch.rand(1, 1, 3, 224, 224)
out = model(src_tensor)

out.shape

NameError: name 'indices' is not defined